# Pipeline ETL
## 1. Ingestion des données brutes depuis les sources dans le namespace bronze
### installation des dependances Geopandas

In [3]:
! pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.7/341.7 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.4/32.4 MB 12.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 11.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType
import os
import zipfile
import geopandas as gpd

def create_bronze_tables(spark):
    """
    Crée les tables bronze dans le catalogue Iceberg avec les meilleures pratiques.
    """
    
    # --- 1. CONFIGURATION DES SCHÉMAS ---
    weather_schema = StructType([
        StructField("time", TimestampType(), True),
        StructField("temp", DoubleType(), True),
         StructField("rhum", DoubleType(), True),
        StructField("prcp", DoubleType(), True),
        StructField("wspd", DoubleType(), True),
        StructField("pres", DoubleType(), True)
    ])

    # --- 2. TAXI TRIPS (Ingestion Parquet) ---
    print("📊 Étape 1/3: Ingestion des trips taxi...")
    trips_raw_df = spark.read.parquet("/home/iceberg/data/raw/nyc_taxi/taxi_trips/*.parquet")
    
    # On utilise writeTo avec des transformations de partitionnement natives (Hidden Partitioning)
    # Cela évite de créer des colonnes "year" et "month" dans la table.
    trips_raw_df.writeTo("iceberg.bronze.taxi_trips_2024") \
        .tableProperty("write.format.default", "parquet") \
        .tableProperty("write.parquet.compression-codec", "snappy") \
        .partitionedBy(F.months("tpep_pickup_datetime")) \
        .createOrReplace()
    
    print(f"✅ Table bronze.taxi_trips_2024 créée.")

    # --- 3. TAXI ZONES (Ingestion Référence) ---
    print("\n🗺️  Étape 2/3: Ingestion des zones taxi...")
    csv_path = "/home/iceberg/data/raw/nyc_taxi/taxi_zones/taxi_zones.csv"
    
    if os.path.exists(csv_path):
        zones_df = spark.read.option("header", "true").csv(csv_path)
    else:
        # Fallback Shapefile -> Spark via Geopandas
        zip_path = "/home/iceberg/data/raw/nyc_taxi/taxi_zones/taxi_zones.zip"
        extract_path = "/tmp/taxi_zones"
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        
        gdf = gpd.read_file(f"{extract_path}/taxi_zones.shp")
        # On ne garde que les colonnes nécessaires pour la couche bronze
        zones_df = spark.createDataFrame(gdf.drop(columns='geometry'))
        # Optionnel: conversion geometry en WKT si nécessaire pour plus tard
        zones_df = zones_df.withColumn("geometry_wkt", F.lit(gdf.geometry.to_wkt().tolist()))

    zones_df.coalesce(1).writeTo("iceberg.bronze.taxi_zones").createOrReplace()
    print(f"✅ Table bronze.taxi_zones créée.")

    # --- 4. WEATHER DATA (Ingestion CSV avec schéma) ---
    print("\n🌤️  Étape 3/3: Ingestion des données météo...")
    
    weather_path = "/home/iceberg/data/raw/nyc_taxi/weather/central_park_weather_2024.csv"
    
    # Utilisation du schéma statique pour plus de robustesse
    weather_df = spark.read \
        .option("header", "true") \
        .schema(weather_schema) \
        .csv(weather_path)
    
    # Nettoyage minimal de la date
    #weather_df = weather_df.withColumn("time", F.to_date("time"))
    weather_df = weather_df.withColumn(
      "time",
       F.to_timestamp("time", "dd/MM/yyyy HH:mm:ss")
     )
    
    # Créer date_ref pour partitionnement
    weather_df = weather_df.withColumn(
       "date_ref",
        F.to_date("time")
    )

    # Vérifier
    weather_df.select("time","date_ref", "temp", "rhum", "prcp","wspd","pres").show(5, truncate=False)
    # Partitionnement par jour (natif Iceberg)
    weather_df.writeTo("iceberg.bronze.weather_2024") \
        .partitionedBy(F.days("date_ref")) \
        .createOrReplace()
    
    print(f"✅ Table bronze.weather_2024 créée.")
    print("\n🎉 Pipeline Bronze terminé avec succès!")

    return True

In [5]:
# On appelle la fonction en lui passant l'objet spark créé dans le notebook 01
create_bronze_tables(spark)

📊 Étape 1/3: Ingestion des trips taxi...


✅ Table bronze.taxi_trips_2024 créée.

🗺️  Étape 2/3: Ingestion des zones taxi...


26/01/06 13:33:17 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
                                                                                

✅ Table bronze.taxi_zones créée.

🌤️  Étape 3/3: Ingestion des données météo...
+-------------------+----------+----+----+----+----+------+
|time               |date_ref  |temp|rhum|prcp|wspd|pres  |
+-------------------+----------+----+----+----+----+------+
|2024-01-01 00:00:00|2024-01-01|5.0 |55.0|0.0 |9.0 |1016.0|
|2024-01-01 01:00:00|2024-01-01|5.0 |55.0|0.0 |9.0 |1016.0|
|2024-01-01 02:00:00|2024-01-01|5.0 |55.0|0.0 |13.0|1016.0|
|2024-01-01 03:00:00|2024-01-01|4.0 |60.0|0.0 |6.0 |1016.0|
|2024-01-01 04:00:00|2024-01-01|4.0 |67.0|0.0 |10.1|1016.0|
+-------------------+----------+----+----+----+----+------+
only showing top 5 rows



[Stage 8:>                                                          (0 + 1) / 1]

✅ Table bronze.weather_2024 créée.

🎉 Pipeline Bronze terminé avec succès!


True

In [7]:
# Lister toutes les tables dans le namespace bronze
spark.sql("SHOW TABLES IN iceberg.bronze").show()

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|   bronze|taxi_trips_2024|      false|
|   bronze|     taxi_zones|      false|
|   bronze|   weather_2024|      false|
+---------+---------------+-----------+



In [8]:
# Voir la structure détaillée de la table taxi_trips
spark.sql("DESCRIBE TABLE EXTENDED iceberg.bronze.taxi_trips_2024").show(truncate=False)

+---------------------+-------------+-------+
|col_name             |data_type    |comment|
+---------------------+-------------+-------+
|VendorID             |int          |NULL   |
|tpep_pickup_datetime |timestamp_ntz|NULL   |
|tpep_dropoff_datetime|timestamp_ntz|NULL   |
|passenger_count      |bigint       |NULL   |
|trip_distance        |double       |NULL   |
|RatecodeID           |bigint       |NULL   |
|store_and_fwd_flag   |string       |NULL   |
|PULocationID         |int          |NULL   |
|DOLocationID         |int          |NULL   |
|payment_type         |bigint       |NULL   |
|fare_amount          |double       |NULL   |
|extra                |double       |NULL   |
|mta_tax              |double       |NULL   |
|tip_amount           |double       |NULL   |
|tolls_amount         |double       |NULL   |
|improvement_surcharge|double       |NULL   |
|total_amount         |double       |NULL   |
|congestion_surcharge |double       |NULL   |
|Airport_fee          |double     

In [9]:
# Compter les lignes par table
tables = ["taxi_trips_2024", "taxi_zones", "weather_2024"]

for t in tables:
    count = spark.table(f"iceberg.bronze.{t}").count()
    print(f"✅ Table {t} : {count} lignes")

# Afficher un échantillon
spark.table("iceberg.bronze.taxi_trips_2024").limit(5).toPandas()

✅ Table taxi_trips_2024 : 41169720 lignes
✅ Table taxi_zones : 263 lignes
✅ Table weather_2024 : 8576 lignes


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-12-01 00:12:27,2024-12-01 00:31:12,1,9.76,1,N,138,33,1,38.0,6.0,0.5,4.72,0.00,1.0,51.97,0.0,1.75
1,2,2024-12-01 00:50:35,2024-12-01 01:24:46,4,20.07,2,N,132,236,2,70.0,0.0,0.5,0.00,6.94,1.0,82.69,2.5,1.75
2,2,2024-12-01 00:18:16,2024-12-01 00:33:16,3,2.34,1,N,142,186,1,15.6,1.0,0.5,4.12,0.00,1.0,24.72,2.5,0.00
3,2,2024-12-01 00:56:13,2024-12-01 01:18:25,1,5.05,1,N,107,80,1,26.8,1.0,0.5,5.00,0.00,1.0,36.80,2.5,0.00
4,1,2024-12-01 00:21:17,2024-12-01 00:37:22,1,4.30,1,N,249,141,1,20.5,3.5,0.5,5.10,0.00,1.0,30.60,2.5,0.00


In [13]:
spark.sql("SHOW TABLES IN iceberg.bronze").show()

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|   bronze|taxi_trips_2024|      false|
|   bronze|     taxi_zones|      false|
|   bronze|   weather_2024|      false|
+---------+---------------+-----------+



In [14]:
# Force Spark à lister les tables du catalogue REST
spark.sql("SHOW TABLES IN iceberg.bronze").show()

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|   bronze|taxi_trips_2024|      false|
|   bronze|     taxi_zones|      false|
|   bronze|   weather_2024|      false|
+---------+---------------+-----------+



In [15]:
spark.sql("SHOW TABLES IN iceberg.bronze").show()

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|   bronze|taxi_trips_2024|      false|
|   bronze|     taxi_zones|      false|
|   bronze|   weather_2024|      false|
+---------+---------------+-----------+



In [17]:
# Afficher les 5 premières lignes de la table Silver
spark.table("iceberg.bronze.weather_2024").limit(5).toPandas()


,time,temp,rhum,prcp,wspd,pres,date_ref
0,2024-01-02 00:00:00,3.0,76.0,0.0,7.0,1018.0,2024-01-02
1,2024-01-02 01:00:00,2.0,79.0,0.0,7.0,1018.0,2024-01-02
2,2024-01-02 02:00:00,2.0,75.0,0.0,7.0,1018.0,2024-01-02
3,2024-01-02 03:00:00,3.0,72.0,0.0,9.0,1018.0,2024-01-02
4,2024-01-02 04:00:00,2.0,69.0,0.0,13.0,1018.0,2024-01-02
